In [1]:
%reset -f
%matplotlib agg
%load_ext watermark

In [2]:
#%matplotlib --list

# Wasserhaushalt Weißache Bergen, Niederschlag vs. Abfluss

**Literaturverzeichnis:**
* [1] Abfluss WWA-TS "Weiße Ache Bergen", Messstellennr. 18465600
    - URL: https://www.gkd.bayern.de/de/fluesse/abfluss/bayern/bergen-18465600
* [2] Niederschlag DWD "Siegsdorf-Maria Eck", Stationsnr. 04697
    - URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/more_precip/historical/tageswerte_RR_04697_19510101_20211231_hist.zip
    - URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/more_precip/recent/tageswerte_RR_04697_akt.zip
* Wasserbedarf Wald 320 l/m² Buche, 390 l/m² Fichte:
  - [3] Planzen Ökologie, Gehard Lerch, Akademie Verlag, 1991, Seite 127
  - [4] https://naturwald-akademie.org/waldwissen/wissenschaft-und-politik-fuer-den-wald/wie-viel-wasser-braucht-der-wald/
  - [5] https://www.lwf.bayern.de/mam/cms04/boden-klima/dateien/a66-wasserverbrauch-von-waeldern.pdf 
  - [6] https://www.waldwissen.net/de/lebensraum-wald/waldboden/wald-und-gesamtwasserhaushalt


In [3]:
from datetime import datetime
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import FuncFormatter

import util as ut

%watermark -iv

numpy     : 1.23.3
pandas    : 1.5.0
matplotlib: 3.6.1
ipywidgets: 8.0.2



In [4]:
# pd.set_option("display.max_rows", 20)

In [5]:
date_n, rs = "MESS_DATUM", "RS"
filenames = [
    # 1951 - 2021
    "../weisse_ache/niederschlag/tageswerte_RR_04697_19510101_20211231_hist/"
    "produkt_nieder_tag_19510101_20211231_04697.txt",
    # 2021-04-21 - 2022-10-22
    "../weisse_ache/niederschlag/tageswerte_RR_04697_akt/"
    "produkt_nieder_tag_20210421_20221022_04697.txt",
]

dfn = (
    pd.concat([ut.dwd_read_csv(f) for f in filenames])
    .set_index(date_n)
    .pipe(lambda x: x.groupby(x.index).first())  # drop duplicates
)
dfn.tail(2)

,STATIONS_ID,QN_6,RS,RSF,SH_TAG,NSH_TAG,eor
MESS_DATUM,,,,,,,
2022-10-21,4697,1,3.8,6,0.0,0.0,eor
2022-10-22,4697,1,6.5,6,0.0,0.0,eor


In [6]:
# Datenquelle: Bayerisches Landesamt für Umwelt, www.lfu.bayern.de
date_a, m3 = "Datum", "Mittelwert"
filenames = [
    # 1976 - 2021
    "../weisse_ache/fluesse-abfluss/06223b6dc4edfd31a7a2ed74d4fc938b/fluesse-abfluss/"
    "18465600_beginn_bis_31.12.2021_tmw.csv",
    # 2022 Jan - Sep
    "../weisse_ache/fluesse-abfluss/23772c4a005fda1d0a71c55f5544faa1/fluesse-abfluss/"
    "18465600_01.01.2022_30.09.2022_tmw.csv",
    # 2022 Okt
    "../weisse_ache/fluesse-abfluss/23772c4a005fda1d0a71c55f5544faa1/fluesse-abfluss/"
    "18465600_01.10.2022_22.10.2022_tmw.csv",
    # 2022 23. Okt
    "../weisse_ache/fluesse-abfluss/23772c4a005fda1d0a71c55f5544faa1/fluesse-abfluss/"
    "18465600_23.10.2022_tmw.csv",
]
dfa = (
    pd.concat([ut.hnd_read_csv(f, n_header_lines=10) for f in filenames])
    .set_index(date_a)
    .iloc[2:]  # removes 1956 values
)

dfa.tail(2)

,Mittelwert,Maximum,Minimum,Prüfstatus
Datum,,,,
2022-10-22,0.484,0.837,0.336,Rohdaten
2022-10-23,0.519,0.692,0.384,Rohdaten


In [7]:
assert dfa[dfa["Prüfstatus"] != "Geprueft"].index[0] >= pd.Timestamp(2021, 1, 1)

In [8]:
# dmask = lambda x: (pd.Timestamp(1977, 1, 1) <= x.index) & (
#    x.index < pd.Timestamp(2022, 1, 1)
# )

df_orig = pd.merge(dfn, dfa, left_index=True, right_index=True)[
    [rs, m3]
]  # .loc[dmask, [rs, m3]]

In [9]:
# Abflussjahr vom 01.11. bis zum 31.10., see
# https://www.gkd.bayern.de/de/fluesse/abfluss/bayern/bergen-18465600/statistik

FREQ = "AS-NOV"
DATE = "year"
diff = "diff"


def calc(freq=FREQ):
    df = (
        df_orig.groupby(pd.Grouper(freq=freq, closed="left"))
        .apply(
            lambda x: pd.Series(
                {
                    rs: (vrs := x[rs].mean() * 365.25 * 1e-3 * 18.5 * 1e6),
                    m3: (vm3 := x[m3].mean() * 3600 * 24 * 365.25),
                    diff: vrs - vm3,
                    "N_group": len(x),
                    "N_nan": np.isnan(x[rs]).sum() + np.isnan(x[m3]).sum(),
                    "start": x.index[0],
                    "stop": x.index[-1],
                }
            )
        )
        .pipe(lambda x: x.set_index(x.index.strftime("%Y").astype("i") + 1))
    )
    return df


pd.concat([calc().head(2), calc().tail(2)])

,RS,Mittelwert,diff,N_group,N_nan,start,stop
1977,3.760849e+07,2.556615e+07,1.204234e+07,365,0,1976-11-01,1977-10-31
1978,3.200285e+07,2.175451e+07,1.024834e+07,365,0,1977-11-01,1978-10-31
2021,2.721918e+07,2.194516e+07,5.274025e+06,365,0,2020-11-01,2021-10-31
2022,2.537908e+07,1.853991e+07,6.839165e+06,356,0,2021-11-01,2022-10-22


In [10]:
# %matplotlib widget
# plt.close("all")

df = calc("AS-NOV")
roll = df[[rs, m3, diff]].rolling(window=20, center=True).apply(np.nanmean)
fig, ax = plt.subplots(figsize=(12.5, 7), clear=True, layout="tight", num=" ")  # 12.5
ax.grid()

for key, offset, width, color, color2, label in zip(
    [rs, m3, diff],
    [-0.2, 0.2, 0],
    [0.4, 0.4, 0.8],
    ["tab:blue", "tab:green", "0.4"],
    ["darkblue", "darkgreen", "k"],
    [
        "Jahresgebietsniederschlag",
        "Jahresabfluss",
        "Differenz: Niederschlag - Abfluss",
    ],
):
    # ax.plot(df.index, df[key], color=color, label=label)
    ax.bar(df.index + offset, df[key], color=color, alpha=0.9, width=width, label=label)
    ax.plot(roll.index, roll[key], color=color2, lw=3)

ax.set_ylim(0, None)
ax.set_xlim(1976 + 0.4, 2022 + 0.5)
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{x/1e6:g}"))
ax.axhline(1e6, color="k", alpha=0.8)
ax.set_ylabel("Jahreswassermenge [$Millionen\;m^3$]")
ax.set_xlabel("Datum")
ax.legend().get_frame().set_alpha(0.8)

ax2 = ax.twinx()
ax2.set_ylabel("Jahresniederschlag $[l/m^2]$")
ax2.set_ylim(ax.get_ylim())
ax2.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{x*1e3/18.5e6:.0f}"))
ax2.format_coord = lambda x, y: f"x={x:.0f}, y=({y/1e6:.1f}, {y/18.5e3:.0f})"

fig.savefig("niederschlag-abfluss.svg") # , dpi=600)

In [11]:
%%html
<img src="niederschlag-abfluss.svg" width="800" />

In [12]:
prc = df[diff] / df[rs]

fix, ax = plt.subplots(figsize=(10, 6), clear=True, layout="tight", num=" ")
ax.bar(df.index, prc, color="0.4", alpha=0.8, width=0.9)
ax.plot(
    df.index,
    prc.rolling(window=20, center=True).apply(np.nanmean),
    "k",
    lw=3,
)
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{x*1e2:g}%"))
ax.grid()
ax.set_xlim(1977 + 0.4, 2020 + 0.5)
ax.set_ylim(0, None)
ax.set_ylabel("Nicht abgeflossener Niederschlag")
ax.set_xlabel("Datum")

fig.savefig("prc_niederschlag-abfluss.svg") #, dpi=600)

In [13]:
%%html
<img src="prc_niederschlag-abfluss.svg" width="800" />

# Wasserbedarf Vegitation

[bayernatlas_einzugsgebiet_marked_small.jpg](bayernatlas_einzugsgebiet_marked_small.jpg) zeigt das Einzugsgebiet und dessen prozentuale Bewaldung.


In [14]:
%%html
<img src="bayernatlas_einzugsgebiet_marked_small.jpg" width="600">

In [15]:
perc_forest = 91 / (28 + 91)
f"{perc_forest*1e2:.1f}% bewaldet"

'76.5% bewaldet'

In [16]:
beech = 320  # l/m² [3,4,5,6]
spruce = 390  # l/m² [3,4,5,6]
water_forest = np.mean([spruce, beech]) / 1e3 * 18.5e6 * perc_forest
f"Wasserbedarf Wald im Einzugsgebiet {water_forest/1e6:.2f} Mio m³"

'Wasserbedarf Wald im Einzugsgebiet 5.02 Mio m³'

# Appendix

## Interactive Plot